In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import mltools as ml
import matplotlib.pyplot as plt
%matplotlib inline
import time
from sklearn import *
np.random.seed(0)

/Users/lijian/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/lijian/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/lijian/anaconda3/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0

In [13]:
X = np.genfromtxt("data/X_train.txt",delimiter=None)
Y = np.genfromtxt("data/Y_train.txt",delimiter=None)
Xte = np.genfromtxt('data/X_test.txt',delimiter=None)

#comment this for y_submit
#Xtr, Xva, Ytr, Yva = ml.splitData(X[:10000], Y[:10000], 0.75)

#Build classifiers
class random_forest:
    def predict(self, Xtr, Ytr, Xte):
        rf = ensemble.RandomForestClassifier(
                        n_estimators=500, min_samples_leaf= 4, n_jobs=-1 ,max_depth=13)
        rf.fit(Xtr,Ytr)
        return rf.predict(Xte)

class knn_classifier:
    def predict(self,Xtr,Ytr,Xte):
        # Cut training features
        Xtr_selected = np.ndarray(shape=(Xtr.shape[0],3))
        feature_most_important = [0, 13, 9, 3, 6, 5, 10, 4, 7, 11, 12, 8, 1, 2]
        for feature_idx in range(3):
            feature = feature_most_important[feature_idx]
            Xtr_single_feature = Xtr[:,feature]
            Xtr_selected[:,feature_idx] = Xtr_single_feature

        # Construct the knn classifier
        knn = ml.knn.knnClassify(Xtr_selected, Ytr, K=85)

        # Cut testing features
        Xte_selected = np.ndarray(shape=(Xte.shape[0],3))
        for feature_idx in range(3):
            feature = feature_most_important[feature_idx]
            Xte_single_feature = Xte[:,feature]
            Xte_selected[:,feature_idx] = Xte_single_feature
        return knn.predict(Xte_selected)
    
class gradient_boost:
    def predict(self,Xtr,Ytr,Xte):
        gd = ensemble.GradientBoostingClassifier(n_estimators=4, learning_rate=0.5,
                 max_depth=8, max_leaf_nodes=16, min_samples_split = 2,min_samples_leaf=2)
        gd.fit(Xtr,Ytr)
#         scoreTr = gd.score(Xtr, Ytr)
#         scoreVa = gd.score(Xva, Yva)
        return gd.predict(Xte)

In [15]:
classifier_list = [(random_forest(), 20),
                   (gradient_boost(), 11),
                   (knn_classifier(), 1)
                  ]
Yhat = []
for classifier in classifier_list:
    buf= classifier[0].predict(X,Y,Xte)
    for weight in range(classifier[1]):
        Yhat.append(buf)
    print("finished")
    

Yhat_average = np.mean(np.array(Yhat), axis=0)
# print("roc_auc_score:", metrics.roc_auc_score(Yva, Yhat_average))
np.savetxt("Y_submit.txt",
           np.vstack( (np.arange(len(Yhat_average)) , Yhat_average) ).T,
           '%d, %.2f',header='ID,Prob1',comments='',delimiter=',');

KeyboardInterrupt: 